In [8]:
import os, sys
import json
import cv2
import numpy as np
lang = "zh"
mode = "val"
label_mode = "test"

# mode = "train"
# label_mode = "train"

max_norm_size = 1000
set_path = "./data/XFUN_v1.0_data/" + lang + "/"
label_path = set_path + lang + "." + mode + ".json"
img_set_path = set_path + mode + "/"
label_set_dict = {}

save_label_path = set_path + label_mode + ".txt"
save_label_bbox_path = set_path + label_mode + "_box.txt"
save_label_image_path = set_path + label_mode + "_image.txt"
save_label_map_path = set_path + "labels.txt"
foutl = open(save_label_path, "wb")
foutb = open(save_label_bbox_path, "wb")
fouti = open(save_label_image_path, "wb")
foutlm = open(save_label_map_path, "wb")

def get_norm_resize_bbox(box, img_height, img_width, max_norm_size=1000):
    im_size_max = max(img_height, img_width)
    im_scale = float(max_norm_size) / float(im_size_max)
    box = np.array(box) * im_scale
    box = [int(np.round(i)) for i in box]
    return box

with open(label_path, "rb") as fin:
    lines = fin.readlines()
    label_data = json.loads(lines[0])['documents']
    sample_num = len(label_data)
    for sno in range(sample_num):
        img_id = label_data[sno]['id']
        document = label_data[sno]['document']
        bbox_num = len(document)
        img_path = img_set_path + img_id + ".jpg"
        img = cv2.imread(img_path)
        img_height, img_width = img.shape[0:2]
        for bno in range(bbox_num):
            bbox = document[bno]['box']
            label = document[bno]['label']
            text = document[bno]['text']
            if label == "other":
                label = "O"
            otstr = text + "\t" + label + "\n"
            foutl.write(otstr.encode('utf-8'))
            
            otstr = "%s\t%d %d %d %d\t%d %d\t%s\n" % (
                text, bbox[0], bbox[1], bbox[2], bbox[3],
                img_width, img_height, img_path)
            fouti.write(otstr.encode('utf-8'))

            bbox = get_norm_resize_bbox(bbox, img_height, img_width, max_norm_size)
            otstr = "%s\t%d %d %d %d\n" % (
                text, bbox[0], bbox[1], bbox[2], bbox[3])
            foutb.write(otstr.encode('utf-8'))

            label_set_dict[label] = 0
        otstr = "\n"
        foutl.write(otstr.encode('utf-8'))
        foutb.write(otstr.encode('utf-8'))
        fouti.write(otstr.encode('utf-8'))

for label in label_set_dict:
    otstr = "%s\n" % label
    foutlm.write(otstr.encode('utf-8'))
foutl.close()
foutb.close()
fouti.close()
foutlm.close()
print("ok")

Corrupt JPEG data: premature end of data segment


ok
